1.Scrapping data from wbesite

1. Importing libraries essetial to scrapping

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

2. Pointing url to fetch data from

In [50]:
url = 'https://tepbac.com/gia-thuy-san/cat/2-thuy-san/'
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

3. Locating tags where the data in relevant fetching into a dataframe

In [45]:
markets = []
listings = soup.find('div', {'class': 'bg-white rounded'}).find_all('a', {'class': 'h6'})

In [46]:
print(listings)

[<a class="h6" href="https://tepbac.com/gia-thuy-san/history/41/tom-su-loai-20-con-tai-ao.htm" title="Giá {_SPECIES.title">Tôm sú loại 20 con tại ao</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/history/7/tom-su-30-con-ao.htm" title="Giá {_SPECIES.title">Tôm sú 30 con (ao)</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/history/37/tom-su-40-con-ao.htm" title="Giá {_SPECIES.title">Tôm sú 40 con (ao)</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/history/52/tom-dat-ca-mau.htm" title="Giá {_SPECIES.title">Tôm đất Cà Mau</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/history/50/cua-gach.htm" title="Giá {_SPECIES.title">Cua gạch</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/history/51/cua-thit.htm" title="Giá {_SPECIES.title">Cua thịt</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/history/39/tom-the-80conkg-tai-ao.htm" title="Giá {_SPECIES.title">Tôm thẻ (80con/kg) tại ao</a>, <a class="h6" href="https://tepbac.com/gia-thuy-san/hi

In [52]:
print(soup.prettify())

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="vi" ng-app="" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#">
 <head profile="http://gmpg.org/xfn/11">
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <title>
   Bảng giá Thủy sản | Giá thủy sản - Tép Bạc
  </title>
  <meta content="giá Thủy sản,bảng giá Thủy sản,giá cả, thủy sản, thị trường thủy sản, tình hình giá cả thủy sản, giá cả thủy sản" itemprop="keywords" name="keywords">
   <meta content="noodp,index,follow" name="robots">
    <meta content="1 days" name="revisit-after"/>
    <meta content="Bảng giá Thủy sản | Giá thủy sản - Tép Bạc" name="title" property="og:title"/>
    <meta content="" itemprop="thumbnailUrl" property="og:image"/>
    <meta content="Thống kê, tổng hợp giá cả Thủy sản cập nhật sự thay đổi liên tục từ khắp các vùng cả nước. Giá cả thị trường thủy sản giá Thủy sản." itemprop="description" name="

4. Fetching data from located tags in the HTML into a proper dataframe.

In [63]:
markets = []
listings = soup.find('div', {'class': 'bg-white rounded'}).find_all('title', {'class': 'h6'})

In [64]:
type(listings)

bs4.element.ResultSet

In [65]:
print(listings)

[]


In [57]:
for item in listings:
    market = {}

    market['specie'] = item.find('a', {'class':'h6'}).getText()
    market['price'] = item.find('td', {'class':'text-right text-danger font-weight-bold'}).getText()
    market['date'] = item.find('td', {'class':'text-right'}).getText()
    market['city'] = item.find('td class', {'a rel':'nofollow'}).getText()
  

    markets.append(market)
  
pricing = pd.DataFrame(markets)
pricing

""


In [61]:
print(markets)

[]


In [59]:
type(market)

dict

In [5]:
pricing['price'] = pricing['price'].astype(float)

5. Create Unit equalizing column

Find the price units that  distorts (dressing percentage) the value of the dafult price of a pig at the gate and equalize it for a price per kilo to into all currencies

In [6]:
treat_price=pricing.loc[(pricing['unit']=='100kg')]
treat_price

,country,date,price,CUR,unit,reference,variation
7,Romania,24 Jul,536.79,RON,100kg,Live,2.0%
15,Canada - Ontario,31 Jul,127.01,CAD,100kg,Carcass,6.1%
16,Canada - Quebec,7 Aug,148.18,CAD,100kg,Carcass,4.9%


In [7]:
treat_price.price

7     536.79
15    127.01
16    148.18
Name: price, dtype: float64

In [8]:
correct_price = treat_price.price.div(100)
correct_price

7     5.3679
15    1.2701
16    1.4818
Name: price, dtype: float64

In [9]:
treat_price2=pricing.loc[(pricing['unit']=='cwt')]
treat_price2

,country,date,price,CUR,unit,reference,variation
20,USA - Iowa/Minnesota,6 Aug,38.81,USD,cwt,Carcass,2.0%


In [10]:
correct_price2 = treat_price2.price.div(50.802)
correct_price2

20    0.763946
Name: price, dtype: float64

In [11]:
treat_price3=pricing.loc[(pricing['CUR']=='GBX')]
treat_price3

,country,date,price,CUR,unit,reference,variation
10,United Kingdom,1 Aug,164.68,GBX,kg,SPP Euro Spec,0.3%


In [12]:
correct_price3 = treat_price3.price.div(100)
correct_price3

10    1.6468
Name: price, dtype: float64

In [13]:
pricing.update(correct_price)

In [14]:
pricing.update(correct_price2)

In [15]:
pricing.update(correct_price3)

In [16]:
pricing

,country,date,price,CUR,unit,reference,variation
0,Belgium - Danis,6 Aug,0.960000,EUR,kg,Live,0%
1,Denmark,6 Aug,10.500000,DKK,kg,Carcass 60%,0%
2,France - MPB,3 Aug,1.293000,EUR,kg,Carcass 56 TMV,0%
3,Germany,5 Aug,1.470000,EUR,kg,Carcass 57%,0%
4,Italy,6 Aug,1.325000,EUR,kg,160/176 kg Live,3.9%
5,Netherlands - Beursprijs,31 Jul,1.050000,EUR,kg,Live,0%
6,Poland,31 Jul,6.441000,PLN,kg,Carcass 57%,2.9%
7,Romania,24 Jul,5.367900,RON,100kg,Live,2.0%
8,Russia,29 Jul,112.200000,RUB,kg,Live,6.8%
9,Spain - Lleida,6 Aug,1.300000,EUR,kg,Live,0%


In [17]:
##4.1 exporting prices for swine into a CSV file

In [18]:
import datetime

current_date = datetime.datetime.now()
filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
pricing.to_csv(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday'+filename+ '.csv'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')

6. Creat Currency equalizing column

6.1 Finding the currency conversions on a website online to keep the values always updated with FX figures

In [19]:
api_url = "https://openexchangerates.org/api/latest.json?app_id=84f0e7f007024e6aa0c5fc52f419b104"
r = requests.get(api_url)

r.raise_for_status()
result = r.json()
rates = result['rates']
print(rates)

{'AED': 3.673, 'AFN': 76.800005, 'ALL': 105.575, 'AMD': 481.616228, 'ANG': 1.795356, 'AOA': 582.5, 'ARS': 72.764767, 'AUD': 1.397302, 'AWG': 1.8002, 'AZN': 1.7025, 'BAM': 1.654608, 'BBD': 2, 'BDT': 84.808663, 'BGN': 1.65944, 'BHD': 0.376966, 'BIF': 1927, 'BMD': 1, 'BND': 1.371218, 'BOB': 6.906511, 'BRL': 5.437325, 'BSD': 1, 'BTC': 8.6472005e-05, 'BTN': 74.908281, 'BWP': 11.739351, 'BYN': 2.451386, 'BZD': 2.016074, 'CAD': 1.33852, 'CDF': 1989, 'CHF': 0.91275, 'CLF': 0.028191, 'CLP': 788.801588, 'CNH': 6.972, 'CNY': 6.9678, 'COP': 3753.810196, 'CRC': 589.781655, 'CUC': 1, 'CUP': 25.75, 'CVE': 94.37, 'CZK': 22.32335, 'DJF': 178, 'DKK': 6.3201, 'DOP': 58.63, 'DZD': 128.540923, 'EGP': 15.959148, 'ERN': 15.002917, 'ETB': 35.4, 'EUR': 0.848259, 'FJD': 2.125, 'FKP': 0.766283, 'GBP': 0.766283, 'GEL': 3.085, 'GGP': 0.766283, 'GHS': 5.77, 'GIP': 0.766283, 'GMD': 51.87, 'GNF': 9615, 'GTQ': 7.701451, 'GYD': 208.982189, 'HKD': 7.75051, 'HNL': 24.899999, 'HRK': 6.331661, 'HTG': 112.346353, 'HUF': 293

In [20]:
usdrates = pd.DataFrame([rates])
usdrates.T.reset_index()

,index,0
0,AED,3.673000
1,AFN,76.800005
2,ALL,105.575000
3,AMD,481.616228
4,ANG,1.795356
...,...,...
166,XPT,0.001036
167,YER,250.350066
168,ZAR,17.637670
169,ZMW,18.305943


6.2 Selecting only the currencys I actually need conversion for the quotations I have in each country

Filtering the ACTUAL CURRENCIES I need (16 only of actual_currencies) from the total FX rates I've have caputred

In [21]:
actual_currency = pricing['CUR']
actual_currency[10]='GBP'
actual_currency

/Users/lstorino/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0     EUR
1     DKK
2     EUR
3     EUR
4     EUR
5     EUR
6     PLN
7     RON
8     RUB
9     EUR
10    GBP
11    UAH
12    ARS
13    BRL
14    BRL
15    CAD
16    CAD
17    CLP
18    COP
19    MXN
20    USD
21    CNY
22    KRW
Name: CUR, dtype: object

In [22]:
actual_currency_list = list(actual_currency)
usd_daily_rate=usdrates[actual_currency_list]
usd_d_rates = usd_daily_rate.T.reset_index()

In [23]:
usd_d_rates

,index,0
0,EUR,0.848259
1,DKK,6.320100
2,EUR,0.848259
3,EUR,0.848259
4,EUR,0.848259
5,EUR,0.848259
6,PLN,3.744080
7,RON,4.105500
8,RUB,73.713000
9,EUR,0.848259


In [24]:
usd_d_rates.rename(columns={'index': 'CUR', 0: 'Rate'},inplace = True)
usd_d_rates

,CUR,Rate
0,EUR,0.848259
1,DKK,6.320100
2,EUR,0.848259
3,EUR,0.848259
4,EUR,0.848259
5,EUR,0.848259
6,PLN,3.744080
7,RON,4.105500
8,RUB,73.713000
9,EUR,0.848259


In [25]:
pricing

,country,date,price,CUR,unit,reference,variation
0,Belgium - Danis,6 Aug,0.960000,EUR,kg,Live,0%
1,Denmark,6 Aug,10.500000,DKK,kg,Carcass 60%,0%
2,France - MPB,3 Aug,1.293000,EUR,kg,Carcass 56 TMV,0%
3,Germany,5 Aug,1.470000,EUR,kg,Carcass 57%,0%
4,Italy,6 Aug,1.325000,EUR,kg,160/176 kg Live,3.9%
5,Netherlands - Beursprijs,31 Jul,1.050000,EUR,kg,Live,0%
6,Poland,31 Jul,6.441000,PLN,kg,Carcass 57%,2.9%
7,Romania,24 Jul,5.367900,RON,100kg,Live,2.0%
8,Russia,29 Jul,112.200000,RUB,kg,Live,6.8%
9,Spain - Lleida,6 Aug,1.300000,EUR,kg,Live,0%


In [26]:
import seaborn as sns

In [27]:
pricing['usd_base_kg'] = pricing.price.div(usd_d_rates.Rate)
pig_price_kg_usd = pricing[['country','date','usd_base_kg']]
pig_price_kg_usd.style.hide_index().highlight_max(color = 'yellow').highlight_min(color = 'lightblue')

country,date,usd_base_kg
Belgium - Danis,6 Aug,1.131730
Denmark,6 Aug,1.661366
France - MPB,3 Aug,1.524299
Germany,5 Aug,1.732961
Italy,6 Aug,1.562023
Netherlands - Beursprijs,31 Jul,1.237829
Poland,31 Jul,1.720316
Romania,24 Jul,1.307490
Russia,29 Jul,1.522120
Spain - Lleida,6 Aug,1.532551


In [28]:
pig_price_kg_usd.style.background_gradient(cmap = 'Greens')

,country,date,usd_base_kg
0,Belgium - Danis,6 Aug,1.131730
1,Denmark,6 Aug,1.661366
2,France - MPB,3 Aug,1.524299
3,Germany,5 Aug,1.732961
4,Italy,6 Aug,1.562023
5,Netherlands - Beursprijs,31 Jul,1.237829
6,Poland,31 Jul,1.720316
7,Romania,24 Jul,1.307490
8,Russia,29 Jul,1.522120
9,Spain - Lleida,6 Aug,1.532551


In [29]:
usd_pig_price_blue = pig_price_kg_usd.sort_values(by = 'usd_base_kg').style.bar(color= 'lightblue')

In [30]:
usd_pig_price_blue

,country,date,usd_base_kg
20,USA - Iowa/Minnesota,6 Aug,0.763946
12,Argentine,24 Jul,0.831584
15,Canada - Ontario,31 Jul,0.948884
16,Canada - Quebec,7 Aug,1.107044
13,Brazil - Paraná,3 Aug,1.129232
0,Belgium - Danis,6 Aug,1.131730
19,Mexico - Sonora,31 Jul,1.134541
14,Brazil - Santa Catarina,6 Aug,1.215671
5,Netherlands - Beursprijs,31 Jul,1.237829
7,Romania,24 Jul,1.307490


In [31]:
import datetime

current_date = datetime.datetime.now()
filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
usd_pig_price_blue.to_excel(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetodayUSDBlueGrade'+filename+ '.xls'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')

In [32]:
pig_price_kg_usd.describe()

,usd_base_kg
count,23.000000
mean,1.646326
std,0.977138
min,0.763946
25%,1.133136
50%,1.522120
75%,1.667909
max,5.301530


8. Export to CSV file for further temporal analisys

In [33]:
import datetime

current_date = datetime.datetime.now()
filename = str(current_date.day)+str(current_date.month)+str(current_date.year)
pig_price_kg_usd.to_csv(str('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetodayUSD'+filename+ '.csv'))
#pricing.to_csv('/Users/lstorino/PythonTests/scrapping/Pig333/data/pricetoday.csv')